# Pick and place with retries and recovery example

This example notebook demonstrates how to react to failures by trying to estimate the pose of an object and using various strategies to recover from detection failures.

In particular, this example covers:

- Behavior tree nodes
    - `Retry` with max retry count
    - `Retry` with `recovery` option
    - `Fallback`
    - `Selector` with `Decorators`
    - `Fail`

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect to your solution and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world
simulator = solution.simulator
pose_estimators = solution.pose_estimators

enable_gripper = skills.ai.intrinsic.enable_gripper
estimate_and_update_pose = skills.ai.intrinsic.estimate_and_update_pose
move_robot = skills.ai.intrinsic.move_robot

Then, create the required skill instances:

In [ ]:
enable_gripper = enable_gripper(
    clear_faults=True, gripper=resources.picobot_gripper)

move_home = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

move_view_pose_left = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_left,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

move_view_pose_right = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_right,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

# Moves the building block in the belief world to its estimated position from
# camera image of the simulated/real world. Fails if the object cannot be found.
estimate_pose = estimate_and_update_pose(
    pose_estimator=pose_estimators.building_block_ml_estimator,
    object=world.building_block0,
    camera=resources.wrist_camera,
)

## Simple retry to ensure successful detection of object

First, we create and run a behavior tree which uses a `Retry` node to retry the `estimate_pose` skill for up to a given number of times. If the skill were to fail (e.g. because of instable lighting conditions) it would be retried up to the given limit:

In [ ]:
world.reset()

estimate_with_retry = bt.Retry(
    max_tries=3, child=bt.Task(action=estimate_pose, name="Estimate pose")
)

retry_3_times = bt.BehaviorTree(
    name="Retry 3 times",
    root=bt.Sequence(
        [
            bt.Task(action=move_view_pose_left, name="Move to view pose"),
            estimate_with_retry,
        ]
    ),
)

executive.run(retry_3_times)

## Retry with recovery

If an iteration of `Retry` fails, you can set a `recovery` node that will be executed before each retry:

In [ ]:
world.reset()
simulator.reset()

# Recovery: Moving to a pose from which the object is visible will help!
move_to_view_pose_recovery = bt.Task(
    action=move_view_pose_left, name="Move to view pose"
)

estimate_and_move_with_recovery = bt.Retry(
    max_tries=2,
    child=bt.Task(action=estimate_pose, name="Estimate pose"),
    recovery=move_to_view_pose_recovery,
)

retry_with_recovery = bt.BehaviorTree(
    name="Retry with recovery",
    root=bt.Sequence(
        [
            # Initially move to a position from which object is not visible.
            bt.Task(action=move_home, name="Move to bad position"),
            estimate_and_move_with_recovery,
        ]
    ),
)

executive.run(retry_with_recovery)

## Retry with counter

The following shows an advanced recovery strategy: We try `estimate_pose` four times, executing a different recovery attempt after each try by using the `retry_counter` of the `Retry` node in combination with a `Selector` node and `Decorators`.

In [ ]:
world.reset()
simulator.reset()

# Recovery attempt 1: Enabling the gripper will *not* help.
recovery1 = bt.Task(action=enable_gripper, name="Recovery 1: Gripper")

# Recovery attempt 2: Moving to the wrong view pose will *not* help.
recovery2 = bt.Task(action=move_view_pose_right, name="Recovery 2: Move R")

# Recovery attempt 3: Moving to the correct view pose will help.
recovery3 = bt.Task(action=move_view_pose_left, name="Recovery 3: Move L")

# The Fallback node executes a sequence of actions up to the first action that succeeds.
# Because we setup the last action to always fail and we use a Retry node, we effectively
# retry `estimate_pose` until it succeeds.
estimate_with_recovery = bt.Fallback(
    [
        bt.Task(action=estimate_pose, name="Try estimate pose"),
        bt.Sequence(
            [
                # The Selector node executes the first action whose decorator condition evaluates
                # to true (see below).
                bt.Selector([recovery1, recovery2, recovery3]),
                # Failing the Fallback node causes the parent Retry node to retry.
                bt.Fail(failure_message="Recovery attempted, triggering retry"),
            ]
        ),
    ]
)
estimate_and_recover = bt.Retry(max_tries=4, child=estimate_with_recovery)

# Set up the decorator conditions based on the Retry counter so that in every Retry iteration we
# use a different retry strategy.
recovery1.set_decorators(
    bt.Decorators(bt.Blackboard(f"{estimate_and_recover.retry_counter} == 0"))
)
recovery2.set_decorators(
    bt.Decorators(bt.Blackboard(f"{estimate_and_recover.retry_counter} == 1"))
)
recovery3.set_decorators(
    bt.Decorators(bt.Blackboard(f"{estimate_and_recover.retry_counter} == 2"))
)

retry_with_counter = bt.BehaviorTree(
    name="Retry with counter",
    root=bt.Sequence(
        [
            # Initially move to a position from which object is not visible.
            bt.Task(action=move_home, name="Move to bad position"),
            estimate_and_recover,
        ]
    ),
)

executive.run(retry_with_counter)